In [63]:
# Carregando libs
import pandas as pd
from datetime import datetime

In [55]:
# Base alvo
df_pb = pd.read_csv('.//baseAtaque//pb-vacina_150122_adj.csv',sep=';', low_memory=False)
db_vacina = df_pb.copy()

In [56]:
# index
db_vacina.dropna()
db_vacina = df_pb.copy()
db_vacina = db_vacina.loc[db_vacina.vacina_descricao_dose == '1ª Dose']
db_vacina = db_vacina.set_index([
    'paciente_dataNascimento', 
    'paciente_enumSexoBiologico', 
    'paciente_racaCor_valor', 
    'paciente_endereco_uf',
    'paciente_endereco_nmMunicipio'
])
db_vacina = db_vacina.sort_index()

In [5]:
# Bases externa
df_prouni = pd.read_csv('ProuniRelatorio2020.csv', sep=';',encoding= 'unicode_escape')

In [6]:
colunas = [
    'ANO_CONCESSAO_BOLSA',
    'CODIGO_EMEC_IES_BOLSA',
    'NOME_IES_BOLSA',
    'MUNICIPIO',
    'CAMPUS',
    'TIPO_BOLSA',
    'MODALIDADE_ENSINO_BOLSA',
    'NOME_CURSO_BOLSA',
    'NOME_TURNO_CURSO_BOLSA',
    'CPF_BENEFICIARIO',
    'SEXO_BENEFICIARIO',
    'RACA_BENEFICIARIO',
    'DATA_NASCIMENTO',
    'BENEFICIARIO_DEFICIENTE_FISICO',
    'REGIAO_BENEFICIARIO',
    'UF_BENEFICIARIO',
    'MUNICIPIO_BENEFICIARIO',
]
df_prouni = df_prouni[colunas]
df_prouni.RACA_BENEFICIARIO = df_prouni.RACA_BENEFICIARIO.str.upper()
df_prouni.loc[df_prouni.RACA_BENEFICIARIO == 'IND¡GENA', 'RACA_BENEFICIARIO'] = 'INDIGENA'
df_prouni.DATA_NASCIMENTO = pd.to_datetime(df_prouni.DATA_NASCIMENTO)
df_prouni = df_prouni[df_prouni['UF_BENEFICIARIO'] == 'PB']

In [7]:
df_prouni = df_prouni[df_prouni['UF_BENEFICIARIO'] == 'PB']

In [59]:
# script de ataque
from datetime import date 
def calcIdade(birthDate):
    y = int(birthDate[0:4])
    m = int(birthDate[5:7])
    d = int(birthDate[8:])
    bd = date(y, m, d)
    days_in_year = 365.2425
    age = int((date.today() - bd).days / days_in_year) 
    return str(age)
def normDate(date):
    return date.strftime("%Y-%m-%d")
def normRaca(raca):
    return raca.upper()

# script de ataque
identificados = []
contador_identificado = 0
contador_nao_identificado = 0

for idx, r_prouni in df_prouni.iterrows():
    #quasi-identifiers
    # print(r_prouni)
    sexo = r_prouni[10]
    raca = normRaca(r_prouni[11])
    dNasc = normDate(r_prouni[12])
    uf = r_prouni[15]
    cidade = r_prouni[16]
    
    try:
        idx = (dNasc, sexo, raca, uf, cidade)
        ataque = db_vacina.loc[idx]
        pessoa = ataque.copy()
    except KeyError:
        pessoa = []
    
    if len(pessoa) > 0 and len(pessoa) <= 5:
        pessoa['cpf'] = r_prouni[9]
        pessoa['ies'] = r_prouni[2]
        pessoa['curso'] = r_prouni[7]
        
        if len(identificados) == 0:
            identificados = pessoa
        else:
            identificados = pd.concat([identificados,pessoa])
        
        contador_identificado += 1
    else:
        contador_nao_identificado += 1

print('Tota de registros processados:', contador_identificado + contador_nao_identificado)
print('Identificados:', contador_identificado)
print('Não identificados:', contador_nao_identificado)
total = contador_identificado + contador_nao_identificado
persentual = round((contador_identificado / total) * 100, 3)
print('taxa de identificação:', persentual, '%')

C:\Users\Anderson\AppData\Local\Temp/ipykernel_14116/1829838329.py:32: PerformanceWarning: indexing past lexsort depth may impact performance.
  ataque = db_vacina.loc[idx]


Tota de registros processados: 2763
Identificados: 1058
Não identificados: 1705
taxa de identificação: 38.292 %


In [54]:
ataque.vacina_grupoAtendimento_nome

paciente_dataNascimento  paciente_enumSexoBiologico  paciente_racaCor_valor
1998-08-08               F                           PARDA                              Pessoas de 18 a 64 anos
                                                     PARDA                              Pessoas de 18 a 64 anos
                                                     PARDA                              Pessoas de 18 a 64 anos
                                                     PARDA                              Pessoas de 18 a 64 anos
                                                     PARDA                              Pessoas de 18 a 64 anos
                                                     PARDA                                        Enfermeiro(a)
                                                     PARDA                                               Outros
                                                     PARDA                              Pessoas de 18 a 64 anos
                            

In [128]:
# Base alvo anonimizada
df_pb = pd.read_csv('.//baseAtaque//pb-vacina_anom20.csv',sep=';', low_memory=False, encoding="ISO-8859-1")
db_vacina = df_pb.copy()
db_vacina.dropna()
db_vacina = df_pb.copy()
db_vacina = db_vacina.loc[db_vacina.vacina_descricao_dose == '1Âª Dose']
db_vacina = db_vacina.set_index([
    'paciente_dataNascimento', 
    'paciente_enumSexoBiologico', 
    'paciente_racaCor_valor', 
    'paciente_endereco_uf',
    'paciente_endereco_nmMunicipio'
])
db_vacina = db_vacina.sort_index()

In [104]:
# index


In [101]:
from datetime import datetime

NIVEL3 = {}
NIVEL4 = {}
NIVEL5 = {}
NIVEL6 = '[1890-01-01, 2021-05-30['

NIVEL3[(date(1890,1,1), date(1920,1,1))] = '[1890-01-01, 1920-01-01['
NIVEL3[(date(1920,1,1), date(1940,1,1))] = '[1920-01-01, 1940-01-01['
NIVEL3[(date(1940,1,1), date(1960,1,1))] = '[1940-01-01, 1960-01-01['
NIVEL3[(date(1960,1,1), date(1980,1,1))] = '[1960-01-01, 1980-01-01['
NIVEL3[(date(1980,1,1), date(2000,1,1))] = '[1980-01-01, 2000-01-01['
NIVEL3[(date(2000,1,1), date(2022,5,30))] = '[2000-01-01, 2021-05-30['

NIVEL4[(date(1890,1,1), date(1940,1,1))] = '[1890-01-01, 1940-01-01['
NIVEL4[(date(1940,1,1), date(1980,1,1))] = '[1940-01-01, 1980-01-01['
NIVEL4[(date(1980,1,1), date(2022,5,30))] = '[1980-01-01, 2021-05-30['

NIVEL5[(date(1890,1,1), date(1980,1,1))] = '[1890-01-01, 1980-01-01['
NIVEL5[(date(1980,1,1), date(2022,5,30))] = '[1980-01-01, 2021-05-30['

In [139]:
def calcIdade(birthDate):
    days_in_year = 365.2425
    age = int((date.today() - birthDate.date()).days / days_in_year) 
    return str(age)

def calcInterval(birthDate):
    if birthDate == '*': return birthDate
    for i, f in NIVEL5.keys():
        if i <= birthDate.date() and f >= birthDate.date():
            return NIVEL5[(i, f)]
    
def normRaca(raca):
    return raca.upper()

# script de ataque
identificados = []
contador_identificado = 0
contador_nao_identificado = 0

minimo = 2957439

for idx, r_prouni in df_prouni.iterrows():
    sexo = r_prouni[10]
    raca = normRaca(r_prouni[11])
    #dNasc = calcInterval(r_prouni[12])
    dNasc = '[1890-01-01, 2021-05-30['
    uf = r_prouni[15]
    cidade = r_prouni[16]
    
    try:
        idx = (dNasc, sexo, raca, uf, cidade)
        #print(idx)
        ataque = db_vacina.loc[idx]
        pessoa = ataque.copy()
    except KeyError:
        pessoa = []
    
    if len(pessoa) > 0 and len(pessoa) < minimo: 
        print(len(pessoa))
        minimo = len(pessoa)
    
    if len(pessoa) > 0 and len(pessoa) <= 180:
        pessoa['cpf'] = r_prouni[9]
        pessoa['ies'] = r_prouni[2]
        pessoa['curso'] = r_prouni[7]
        
        if len(identificados) == 0:
            identificados = pessoa
        else:
            identificados = pd.concat([identificados,pessoa])
        
        contador_identificado += 1
    else:
        contador_nao_identificado += 1
        
    #if contador_identificado + contador_nao_identificado == 10: break
        
print('Identificados', contador_identificado)
print('Não identificados', contador_nao_identificado)
total = contador_identificado + contador_nao_identificado
persentual = (contador_identificado / total) * 100
print('taxa de identificação', persentual, '%')

C:\Users\Anderson\AppData\Local\Temp/ipykernel_14116/3864148460.py:33: PerformanceWarning: indexing past lexsort depth may impact performance.
  ataque = db_vacina.loc[idx]


28372
2345
217
171
Identificados 7
Não identificados 2756
taxa de identificação 0.2533478103510677 %


In [135]:
t={}
for idx, line in db_vacina.iterrows():
    if idx[0] in t.keys(): t[idx[0]] += 1
    else: t[idx[0]] = 1

In [136]:
t

{'*': 171129, '[1890-01-01, 2021-05-30[': 2957439}